In [1]:
import numpy as np
from scipy.stats import norm

# Function to calculate call option price using the Black-Scholes formula
def call_price(s, k, t, r, sigma):
    # Calculate d1 and d2
    d1 = (np.log(s/k) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    # Calculate call option price
    return s*norm.cdf(d1) - k*np.exp(-r*t)*norm.cdf(d2)

# Function to calculate put option price using the Black-Scholes formula
def put_price(s, k, t, r, sigma):
    # Calculate d1 and d2
    d1 = (np.log(s/k) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)
    # Calculate put option price
    return k*np.exp(-r*t)*norm.cdf(-d2) - s*norm.cdf(-d1)

# Function to calculate implied volatility using the Newton-Raphson method
def calculate_IV(s, k, t, r, option_type, market_price):
    sigma = 0.5
    for i in range(100):
        if option_type == 'CE':
            price = call_price(s, k, t, r, sigma)
        else:
            price = put_price(s, k, t, r, sigma)
        vega = s*norm.pdf((np.log(s/k) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t)))*np.sqrt(t)
        if abs(price - market_price) < 0.05:
            break
        if vega == 0:
            vega = 0.0001
        sigma = sigma - (price - market_price)/vega
    return sigma

# Function to calculate option Greeks (Delta, Gamma, Vega, Theta, Rho)
def option_greeks(s, k, t, r, sigma, option_type):
    d1 = (np.log(s/k) + (r + sigma**2/2)*t)/(sigma*np.sqrt(t))
    d2 = d1 - sigma*np.sqrt(t)

    if option_type == 'CE':
        delta = norm.cdf(d1)
        gamma = norm.pdf(d1)/(s*sigma*np.sqrt(t))
        vega = (s*norm.pdf(d1)*np.sqrt(t))/100
        theta = -((s*norm.pdf(d1)*sigma)/(2*np.sqrt(t)))/365
        rho = k*t*np.exp(-r*t)*norm.cdf(d2)/100
    else:
        delta = norm.cdf(d1) - 1
        gamma = norm.pdf(d1)/(s*sigma*np.sqrt(t))
        vega = (s*np.exp(-r*t)*norm.pdf(d1)*np.sqrt(t))/100
        theta = -((s*norm.pdf(d1)*sigma)/(2*np.sqrt(t)))/365
        rho = -k*t*np.exp(-r*t)*norm.cdf(-d2)/100

    return round(delta, 3), round(gamma, 5), round(vega, 2), round(theta, 2), round(rho, 3)

# Given parameters
s = 19778.30
k = 19800
t = 1/365
r = 6.835/100
market_priceCE = 43.7
market_pricePE = 70.5

# Calculate implied volatility for call and put options
sigmace = calculate_IV(s, k, t, r, 'CE', market_priceCE)
sigmape = calculate_IV(s, k, t, r, 'PE', market_pricePE)

# Print results
print("Implied Volatility (Call Option):", sigmace)
print("Implied Volatility (Put Option):", sigmape)
print("Call Option Price with Implied Volatility:", call_price(s, k, t, r, sigmace))
print("Put Option Price with Implied Volatility:", put_price(s, k, t, r, sigmape))
print("Option Greeks (Call Option):", option_greeks(s, k, t, r, sigmace, 'CE'))
print("Option Greeks (Put Option):", option_greeks(s, k, t, r, sigmape, 'PE'))

Implied Volatility (Call Option): 0.12634299211688232
Implied Volatility (Put Option): 0.1478316327980606
Call Option Price with Implied Volatility: 43.70001270982175
Put Option Price with Implied Volatility: 70.50000455478585
Option Greeks (Call Option): (0.447, 0.00302, 4.09, -25.86, 0.241)
Option Greeks (Put Option): (-0.545, 0.00259, 4.1, -30.33, -0.297)
